In [1]:
### install packages
# BiocManager::install("GEOquery")
# BiocManager::install("Mus.musculus")
# BiocManager::install("annotate")
#BiocManager::install('mouse430a2.db')
#BiocManager::install('mouse4302.db')
# BiocManager::install('org.Mm.eg.db'
#BiocManager::install("pd.moex.1.0.st.v1")
#BiocManager::install("moex10stprobeset.db")
#BiocManager::install("moex10sttranscriptcluster.db")

In [2]:
library(matrixStats)
#library(genefilter)
library(edgeR)
#library(lumi)
library(statmod)
library(org.Hs.eg.db)
#library(rgl)
#library(mogene10sttranscriptcluster.db)
#library(moex10sttranscriptcluster.db)
#library(moex10stprobeset.db)
#library(stringr)

# Data plots for selected GEO samples
library(GEOquery)
library(limma)
#library(umap)
# Annotation databases
# library(mouse430a2.db)
# library(mouse4302.db)
library(org.Mm.eg.db)
library(annotate)
#library(Mus.musculus)

Loading required package: limma

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table,

In [26]:
# useful functions

######################################
#This script uses previously found Ensembl to Entrez dictionary and transforms
#Ensembl data into Entrez

Ensembl_to_entrez <- function(dataset, dictionary){
        Entrez_dataset <- NULL
        i <- 0
        for (v in ls(dictionary))
        {
                i <- i+1
                if (length(dictionary[[v]])==1)
                {
                        Entrez_dataset <- rbind(Entrez_dataset, exprs(dataset)[dictionary[[v]],])
                }
                else
                {
                        #print(v)
                        #just summing several ENSEMBLES rows
                        temp_Entrez <- apply(exprs(dataset)[dictionary[[v]],], 2, FUN=sum)
                        Entrez_dataset <- rbind(Entrez_dataset, temp_Entrez)
                }
                rownames(Entrez_dataset)[i] <- v
        }
        
        return(Entrez_dataset)
}


Ensembl_to_entrez_2 <- function(expr_set, dictionary){
        Entrez_dataset <- NULL
        i <- 0
        for (v in ls(dictionary))
        {
                i <- i+1
                if (length(dictionary[[v]])==1)
                {
                        Entrez_dataset <- rbind(Entrez_dataset, expr_set[dictionary[[v]],])
                }
                else
                {
                        #print(v)
                        #just summing several ENSEMBLES rows
                        temp_Entrez <- apply(expr_set[dictionary[[v]],], 2, FUN=sum)
                        Entrez_dataset <- rbind(Entrez_dataset, temp_Entrez)
                }
                rownames(Entrez_dataset)[i] <- v
        }
        
        return(Entrez_dataset)
}


######################################################
#This function creates mice dictionary for Ensembl-Entrez mapping

Ensembl_mouse_dictionary_create <- function(expr_set, lib){
        #expr_set :: set with expression levels
        #lib :: org.Mm dictionary
        #1) First let's create table where every Ensembl has corresponding Entrez ID; 
        #   we will remove samples that do not map or map to several Entrez IDs
        library(org.Mm.eg.db)
        xx <- as.list(lib)   #as.list(org.Mm.egENSEMBL2EG)
        rows <- rownames(expr_set)
        ensembltoentrez <- data.frame(ID=rows, ENTREZID=NA)
        rownames(ensembltoentrez) <- ensembltoentrez$ID #Form a table :: ENS , $ENS, $ENTREZ

        for (i in 1:nrow(expr_set))
        {
                if (length(xx[[rows[i]]])==1)
                {
                        ensembltoentrez[i,2] <- xx[[rows[i]]]
                }
        }

        #Let's remove Ensembl IDs that do not map to any Entrez ID and that map to several Entrez IDs
        ensembltoentrez_noNA <- ensembltoentrez[!is.na(ensembltoentrez$ENTREZID),]
        expr_set <- expr_set[rownames(ensembltoentrez_noNA),] # <- Is this really changed inside the function???
        newrows <- rownames(ensembltoentrez_noNA)
    
        ##TEST
        return(ensembltoentrez_noNA)
        ##
        
#         #2) Let's create library for every Entrez ID to see if any of them are mapped by more than 1 Ensembl ID
#         EntreztoID_rnaseq <- new.env()
#         for (i in 1:length(newrows))
#         {
#                 temp <- strsplit(ensembltoentrez_noNA$ENTREZID[i]," /// ")[[1]]
#                 for (j in 1:length(temp))
#                 {
#                         if (exists(temp[j],EntreztoID_rnaseq))
#                         {
#                                 assign(temp[j], c(get(temp[j],EntreztoID_rnaseq), newrows[i]),
#                                                  envir=EntreztoID_rnaseq)
#                         }
#                         else
#                         {
#                                 assign(temp[j], newrows[i], envir=Entrezto
#ID_rnaseq)
#                         }
#                 }
#         }

#         return(EntreztoID_rnaseq) #environment with EG1 : [ENS1, ENS2, ...], ...
}



# GSE50206

## Mouse

In [7]:
# load series and platform data from GEO
gse_name = "GSE50206"
gpl_name = "GPL10787"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 2 file(s)

GSE50206-GPL10787_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM1330506 = col_double(),
  GSM1330507 = col_double(),
  GSM1330508 = col_double(),
  GSM1330509 = col_double(),
  GSM1330510 = col_double(),
  GSM1330511 = col_double(),
  GSM1330512 = col_double(),
  GSM1330513 = col_double(),
  GSM1330514 = col_double(),
  GSM1330515 = col_double(),
  GSM1330516 = col_double(),
  GSM1330517 = col_double(),
  GSM1330518 = col_double(),
  GSM1330519 = col_double(),
  GSM1330520 = col_double()
)


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL10nnn/GPL10787/annot/GPL10787.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

File stored at: 

/tmp/RtmpYAoca6/GPL10787.soft

Warnin

In [16]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annotation_table)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

[1] "ID"                   "SPOT_ID"              "CONTROL_TYPE"        
 [4] "REFSEQ"               "GB_ACC"               "GENE"                
 [7] "GENE_SYMBOL"          "GENE_NAME"            "UNIGENE_ID"          
[10] "ENSEMBL_ID"           "TIGR_ID"              "ACCESSION_STRING"    
[13] "CHROMOSOMAL_LOCATION" "CYTOBAND"             "DESCRIPTION"         
[16] "GO_ID"                "SEQUENCE"

[1] "Number of samples:" "15"                
[1] "Count of probe_ID:" "55821"             
[1] "Unique probe_ID:" "55821"           


In [24]:
pheno

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_department,contact_institute,contact_address,contact_city,contact_zip/postal_code,contact_country,supplementary_file,data_row_count,cell type:ch1,sample group:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
GSM1330506,Mouse embryonic fibroblast,GSM1330506,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,Mouse embryonic fibroblast,Mus musculus,cell type: Mouse embryonic fibroblast,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330506/suppl/GSM1330506_KT001_m1.txt.gz,55821,Mouse embryonic fibroblast,mouse cells
GSM1330507,Mouse embryonic fibroblast transfected OSKMG on day 3,GSM1330507,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,Mouse embryonic fibroblast transfected OSKMG on day 3,Mus musculus,cell type: Mouse embryonic fibroblast transfected OSKMG on day 3,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330507/suppl/GSM1330507_KT001_m2.txt.gz,55821,Mouse embryonic fibroblast transfected OSKMG on day 3,mouse cells
GSM1330508,SSEA-1 (+)/Nanog-GFP (-) cells on day 7,GSM1330508,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,SSEA-1 (+)/Nanog-GFP (-) cells on day 7,Mus musculus,cell type: SSEA-1 (+)/Nanog-GFP (-) cells on day 7,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330508/suppl/GSM1330508_KT001_m3.txt.gz,55821,SSEA-1 (+)/Nanog-GFP (-) cells on day 7,mouse cells
GSM1330509,SSEA-1 (+)/Nanog-GFP (+) cells on day 7,GSM1330509,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,SSEA-1 (+)/Nanog-GFP (+) cells on day 7,Mus musculus,cell type: SSEA-1 (+)/Nanog-GFP (+) cells on day 7,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330509/suppl/GSM1330509_KT001_m4.txt.gz,55821,SSEA-1 (+)/Nanog-GFP (+) cells on day 7,mouse cells
GSM1330510,SSEA-1 (+)/Nanog-GFP (-) cells on day 11,GSM1330510,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,SSEA-1 (+)/Nanog-GFP (-) cells on day 11,Mus musculus,cell type: SSEA-1 (+)/Nanog-GFP (-) cells on day 11,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330510/suppl/GSM1330510_KT001_m5.txt.gz,55821,SSEA-1 (+)/Nanog-GFP (-) cells on day 11,mouse cells
GSM1330511,SSEA-1 (+)/Nanog-GFP (+) cells on day 11,GSM1330511,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,SSEA-1 (+)/Nanog-GFP (+) cells on day 11,Mus musculus,cell type: SSEA-1 (+)/Nanog-GFP (+) cells on day 11,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330511/suppl/GSM1330511_KT001_m6.txt.gz,55821,SSEA-1 (+)/Nanog-GFP (+) cells on day 11,mouse cells
GSM1330512,SSEA-1 (+)/Nanog-GFP (+) cells on day 15,GSM1330512,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,SSEA-1 (+)/Nanog-GFP (+) cells on day 15,Mus musculus,cell type: SSEA-1 (+)/Nanog-GFP (+) cells on day 15,⋯,Center for iPS Cell Research and Application,Kyoto University,"53 Kawahara-cho, Shogoin, Sakyo-ku",Kyoto,606-8507,Japan,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1330nnn/GSM1330512/suppl/GSM1330512_KT001_m7.txt.gz,55821,SSEA-1 (+)/Nanog-GFP (+) cells on day 15,mouse cells
GSM1330513,SSEA-1 (+)/Nanog-GFP (+) cells on day 20,GSM1330513,Public on Apr 24 2014,Feb 19 2014,Apr 24 2014,RNA,1,

## Human

In [25]:
# load series and platform data from GEO
gse_name = "GSE50206"
gpl_name = "GPL14550"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 2 file(s)

GSE50206-GPL10787_series_matrix.txt.gz

Using locally cached version: /tmp/RtmpYAoca6/GSE50206-GPL10787_series_matrix.txt.gz


── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM1330506 = col_double(),
  GSM1330507 = col_double(),
  GSM1330508 = col_double(),
  GSM1330509 = col_double(),
  GSM1330510 = col_double(),
  GSM1330511 = col_double(),
  GSM1330512 = col_double(),
  GSM1330513 = col_double(),
  GSM1330514 = col_double(),
  GSM1330515 = col_double(),
  GSM1330516 = col_double(),
  GSM1330517 = col_double(),
  GSM1330518 = col_double(),
  GSM1330519 = col_double(),
  GSM1330520 = col_double()
)


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL10nnn/GPL10787/annot/GPL10787.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so w

In [ ]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annotation_table)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

# GSE28688

In [7]:
# load series and platform data from GEO
gse_name = "GSE28688"
gpl_name = "GPL6883"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)

GSE28688_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM710513 = col_double(),
  GSM710514 = col_double(),
  GSM710515 = col_double(),
  GSM710516 = col_double(),
  GSM710517 = col_double(),
  GSM710518 = col_double(),
  GSM710519 = col_double(),
  GSM710520 = col_double(),
  GSM710521 = col_double(),
  GSM710522 = col_double(),
  GSM710523 = col_double(),
  GSM710524 = col_double(),
  GSM710525 = col_double(),
  GSM710526 = col_double()
)


File stored at: 

/tmp/Rtmpz4JCTa/GPL6883.annot.gz



In [8]:
annotation_table <- fData(gset)
colnames(annotation_table)

annot_table_filt <- annotation_table[nchar(annotation_table$GB_ACC) > 0 ,]


print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))
print(c("Unique filt probe_ID:", length(unique(rownames(annot_table_filt)))))

[1] "ID"                    "Gene title"            "Gene symbol"          
 [4] "Gene ID"               "UniGene title"         "UniGene symbol"       
 [7] "UniGene ID"            "Nucleotide Title"      "GI"                   
[10] "GenBank Accession"     "Platform_CLONEID"      "Platform_ORF"         
[13] "Platform_SPOTID"       "Chromosome location"   "Chromosome annotation"
[16] "GO:Function"           "GO:Process"            "GO:Component"         
[19] "GO:Function ID"        "GO:Process ID"         "GO:Component ID"      
[22] "Platform_SEQUENCE"

[1] "Count of probe_ID:" "24526"             
[1] "Unique probe_ID:" "24526"           
[1] "Unique filt probe_ID:" "0"                    


In [9]:
exprs(gset)

,GSM710513,GSM710514,GSM710515,GSM710516,GSM710517,GSM710518,GSM710519,GSM710520,GSM710521,GSM710522,GSM710523,GSM710524,GSM710525,GSM710526
ILMN_1343291,42863.74000,39332.98000,39814.30000,41591.99000,42008.69000,41030.38000,40378.71000,41874.22000,32428.89000,28641.87000,30899.44000,30044.85000,27371.96000,30624.05000
ILMN_1651209,54.49462,57.98246,55.95735,55.63179,52.28317,59.25595,56.55336,61.89759,51.90551,59.91526,54.45332,54.38968,52.69560,50.50858
ILMN_1651228,3546.31900,4038.08700,3750.44300,3449.94100,4687.72800,3816.28500,6380.42400,5006.05200,1634.12200,1435.90300,1313.70000,1705.21500,1513.61900,1582.07600
ILMN_1651229,127.47270,148.36830,135.58570,166.69160,138.25460,155.10520,161.34820,166.07080,97.31795,157.14810,112.58940,173.45100,101.78070,145.43260
ILMN_1651235,54.74259,52.77312,54.18909,56.88378,63.76958,52.36141,59.56725,57.34926,48.60712,46.93316,52.34710,47.17689,50.34439,49.19529
ILMN_1651236,54.74646,56.48574,59.83154,56.35628,60.46148,61.12645,57.39851,59.09838,54.88941,51.78627,63.40659,52.21295,57.60165,52.63198
ILMN_1651237,417.16240,401.77400,223.84420,266.48990,362.93880,345.48330,218.47940,234.94990,692.88190,447.06670,789.39680,191.95230,689.54920,211.85100
ILMN_1651238,40.29019,52.20223,52.57457,48.17575,42.16223,45.37173,48.27447,46.89700,42.68807,42.96605,42.82943,47.97372,42.50072,43.67075
ILMN_1651254,1450.75700,1116.24800,1500.09800,1082.32000,779.71270,703.79110,755.11770,748.53350,1669.34700,3186.20900,473.51100,954.95020,793.83260,1185.51900
ILMN_1651260,47.08601,42.55168,49.08297,49.51261,43.33369,39.23980,45.85206,45.56593,45.80526,45.61488,42.45461,41.52282,40.79204,41.93477


# GSE67462, GSE89455 - NM_annotation (with script)

In [6]:
# load series and platform data from GEO
gse_name = "GSE67462"
gpl_name = "GPL19972"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)

GSE67462_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM1647454 = col_double(),
  GSM1647455 = col_double(),
  GSM1647456 = col_double(),
  GSM1647457 = col_double(),
  GSM1647458 = col_double(),
  GSM1647459 = col_double(),
  GSM1647460 = col_double(),
  GSM1647461 = col_double(),
  GSM1647462 = col_double(),
  GSM1647463 = col_double(),
  GSM1647464 = col_double(),
  GSM1647465 = col_double(),
  GSM1647466 = col_double(),
  GSM1647467 = col_double(),
  GSM1647468 = col_double(),
  GSM1647469 = col_double(),
  GSM1647470 = col_double(),
  GSM1647471 = col_double()
)


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL19nnn/GPL19972/annot/GPL19972.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so

In [23]:
annotation_table <- fData(gset)
colnames(annotation_table)

### split NM columns to Names and Versions
# NM_split <- read.table(text = annotation_table$GB_ACC, sep = ".", colClasses = "character")
# colnames(NM_split) <- c("NM", "version")
# annotation_table <- cbind(annotation_table, NM_split)
# rownames(gset) <- annotation_table$NM

annot_table_filt <- annotation_table[nchar(annotation_table$GB_ACC) > 0 ,]


print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))
print(c("Unique filt probe_ID:", length(unique(rownames(annot_table_filt)))))

[1] "ID"          "GB_ACC"      "Description"

[1] "Count of probe_ID:" "29643"             
[1] "Unique probe_ID:" "29643"           
[1] "Unique filt probe_ID:" "29643"                


In [24]:
expr_set <- exprs(gset)[as.character(annot_table_filt$ID), ]

#check if all rownames in expr table and annot table are equal
if (all(rownames(expr_set)==annot_table_filt$ID))
{
    rownames(expr_set) <- annot_table_filt$GB_ACC
} else {
    print("Problem")
}

In [9]:
expr_set <- rowsum(expr_set, row.names(expr_set))

In [8]:
test <- as.list(org.Mm.egREFSEQ2EG)[1:3]

In [17]:
dictionary <- Ensembl_mouse_dictionary_create(expr_set, org.Mm.egREFSEQ2EG)

In [18]:
dictionary

ID,ENTREZID
<chr>,<lgl>


In [19]:
xx <- as.list(org.Mm.egREFSEQ2EG)   #as.list(org.Mm.egENSEMBL2EG)
rows <- rownames(expr_set)
ensembltoentrez <- data.frame(ID=rows, ENTREZID=NA)
rownames(ensembltoentrez) <- ensembltoentrez$ID

for (i in 1:nrow(expr_set))
{
        if (length(xx[[rows[i]]])==1)
        {
                ensembltoentrez[i,2] <- xx[[rows[i]]]
        }
}

In [20]:
as.list(org.Mm.egREFSEQ2EG)[1:3]

$NM_007376
[1] "11287"

$NP_031402
[1] "11287"

$NM_009591
[1] "11298"

In [21]:
ensembltoentrez

,ID,ENTREZID
,<chr>,<lgl>
NM_001001130.2,NM_001001130.2,NA
NM_001001144.3,NM_001001144.3,NA
NM_001001152.2,NM_001001152.2,NA
NM_001001160.2,NM_001001160.2,NA
NM_001001176.2,NM_001001176.2,NA
NM_001001177.2,NM_001001177.2,NA
NM_001001178.1,NM_001001178.1,NA
NM_001001179.3,NM_001001179.3,NA
NM_001001180.2,NM_001001180.2,NA


In [22]:
rows

[1] "NM_001001130.2" "NM_001001144.3" "NM_001001152.2" "NM_001001160.2"
    [5] "NM_001001176.2" "NM_001001177.2" "NM_001001178.1" "NM_001001179.3"
    [9] "NM_001001180.2" "NM_001001181.3" "NM_001001182.3" "NM_001001183.1"
   [13] "NM_001001184.1" "NM_001001185.3" "NM_001001186.3" "NM_001001187.3"
   [17] "NM_001001295.2" "NM_001001297.2" "NM_001001309.2" "NM_001001320.1"
   [21] "NM_001001321.3" "NM_001001322.2" "NM_001001326.1" "NM_001001332.2"
   [25] "NM_001001333.2" "NM_001001334.2" "NM_001001335.2" "NM_001001444.2"
   [29] "NM_001001445.1" "NM_001001446.3" "NM_001001447.2" "NM_001001450.4"
   [33] "NM_001001451.1" "NM_001001452.1" "NM_001001453.1" "NM_001001454.3"
   [37] "NM_001001488.3" "NM_001001489.3" "NM_001001490.3" "NM_001001491.1"
   [41] "NM_001001492.2" "NM_001001493.2" "NM_001001495.2" "NM_001001496.2"
   [45] "NM_001001559.2" "NM_001001565.2" "NM_001001566.3" "NM_001001602.2"
   [49] "NM_001001650.1" "NM_001001714.1" "NM_001001735.1" "NM_001001738.2"
   [53] "NM_001001792.1" "NM_001001796.4" "NM_001001798.2" "NM_001001803.2"
   [57] "NM_001001804.2" "NM_001001805.2" "NM_001001806.2" "NM_001001807.1"
   [61] "NM_001001808.2" "NM_001001809.2" "NM_001001810.2" "NM_001001880.2"
   [65] "NM_001001881.2" "NM_001001882.3" "NM_001001883.3" "NM_001001884.1"
   [69] "NM_001001885.1" "NM_001001892.2" "NM_001001932.3" "NM_001001979.2"
   [73] "NM_001001980.2" "NM_001001981.3" "NM_001001982.2" "NM_001001983.2"
   [77] "NM_001001984.2" "NM_001001985.3" "NM_001001986.2" "NM_001001999.1"
   [81] "NM_001002004.2" "NM_001002005.2" "NM_001002008.1" "NM_001002011.3"
   [85] "NM_001002012.1" "NM_001002238.2" "NM_001002240.2" "NM_001002241.2"
   [89] "NM_001002267.2" "NM_001002268.3" "NM_001002272.2" "NM_001002764.1"
   [93] "NM_001002775.2" "NM_001002786.2" "NM_001002787.2" "NM_001002791.2"
   [97] "NM_001002842.2" "NM_001002846.2" "NM_001002894.2" "NM_001002895.2"
  [101] "NM_001002896.2" "NM_001002897.3" "NM_001002898.1" "NM_001002900.1"
  [105] "NM_001002927.2" "NM_001002929.4" "NM_001003393.1" "NM_001003405.4"
  [109] "NM_001003664.2" "NM_001003666.2" "NM_001003667.1" "NM_001003668.2"
  [113] "NM_001003670.1" "NM_001003671.1" "NM_001003672.1" "NM_001003685.3"
  [117] "NM_001003717.1" "NM_001003719.2" "NM_001003815.2" "NM_001003817.1"
  [121] "NM_001003824.1" "NM_001003825.2" "NM_001003893.2" "NM_001003898.3"
  [125] "NM_001003899.2" "NM_001003908.1" "NM_001003909.3" "NM_001003910.2"
  [129] "NM_001003911.2" "NM_001003912.1" "NM_001003913.2" "NM_001003915.2"
  [133] "NM_001003916.1" "NM_001003917.3" "NM_001003918.2" "NM_001003919.1"
  [137] "NM_001003920.3" "NM_001003933.2" "NM_001003934.2" "NM_001003939.2"
  [141] "NM_001003947.1" "NM_001003948.2" "NM_001003949.3" "NM_001003950.2"
  [145] "NM_001003951.2" "NM_001003952.1" "NM_001003953.1" "NM_001003955.2"
  [149] "NM_001003960.4" "NM_001003961.4" "NM_001003963.4" "NM_001003971.1"
  [153] "NM_001003972.1" "NM_001003973.1" "NM_001004025.4" "NM_001004061.1"
  [157] "NM_001004062.2" "NM_001004066.3" "NM_001004139.2" "NM_001004140.2"
  [161] "NM_001004141.2" "NM_001004142.2" "NM_001004143.4" "NM_001004144.1"
  [165] "NM_001004145.2" "NM_001004147.3" "NM_001004148.4" "NM_001004149.1"
  [169] "NM_001004150.3" "NM_001004151.2" "NM_001004152.2" "NM_001004153.2"
  [173] "NM_001004154.2" "NM_001004155.2" "NM_001004157.2" "NM_001004158.2"
  [177] "NM_001004159.2" "NM_001004164.2" "NM_001004173.2" "NM_001004174.1"
  [181] "NM_001004176.2" "NM_001004177.2" "NM_001004178.1" "NM_001004180.1"
  [185] "NM_001004182.3" "NM_001004184.3" "NM_001004185.3" "NM_001004187.1"
  [189] "NM_001004190.3" "NM_001004191.3" "NM_001004192.1" "NM_001004193.2"
  [193] "NM_001004194.2" "NM_001004293.1" "NM_001004357.2" "NM_001004359.2"
  [197] "NM_001004361.2" "NM_001004362.2" "NM_001004363.1" "NM_001004364.2"
  [201] "NM_001004365.1" "NM_001004366.1" "NM_001004367.4" "NM_001004435.2"
  [205] "NM_001004436.3" "NM_001004455.2" "NM_001004468.2" "NM_001004721.1"
  [209] "NM_001004761.1" "NM_001004762.3" "NM_001005

In [ ]:
Ensembl_mouse_dictionary_create <- function(expr_set, lib){
        #expr_set :: set with expression levels
        #lib :: org.Mm dictionary
        #1) First let's create table where every Ensembl has corresponding Entrez ID; 
        #   we will remove samples that do not map or map to several Entrez IDs
        xx <- as.list(lib)   #as.list(org.Mm.egENSEMBL2EG)
        rows <- rownames(expr_set)
        ensembltoentrez <- data.frame(ID=rows, ENTREZID=NA)
        rownames(ensembltoentrez) <- ensembltoentrez$ID #Form a table :: ENS , $ENS, $ENTREZ

        for (i in 1:nrow(expr_set))
        {
                if (length(xx[[rows[i]]])==1)
                {
                        ensembltoentrez[i,2] <- xx[[rows[i]]]
                }
        }

        #Let's remove Ensembl IDs that do not map to any Entrez ID and that map to several Entrez IDs
        ensembltoentrez_noNA <- ensembltoentrez[!is.na(ensembltoentrez$ENTREZID),]
        expr_set <- expr_set[rownames(ensembltoentrez_noNA),] # <- Is this really changed inside the function???
        newrows <- rownames(ensembltoentrez_noNA)
    
        ##TEST
        return(ensembltoentrez_noNA)
        ##
        
#         #2) Let's create library for every Entrez ID to see if any of them are mapped by more than 1 Ensembl ID
#         EntreztoID_rnaseq <- new.env()
#         for (i in 1:length(newrows))
#         {
#                 temp <- strsplit(ensembltoentrez_noNA$ENTREZID[i]," /// ")[[1]]
#                 for (j in 1:length(temp))
#                 {
#                         if (exists(temp[j],EntreztoID_rnaseq))
#                         {
#                                 assign(temp[j], c(get(temp[j],EntreztoID_rnaseq), newrows[i]),
#                                                  envir=EntreztoID_rnaseq)
#                         }
#                         else
#                         {
#                                 assign(temp[j], newrows[i], envir=EntreztoID_rnaseq)
#                         }
#                 }
#         }

#         return(EntreztoID_rnaseq) #environment with EG1 : [ENS1, ENS2, ...], ...
}

In [ ]:
Entrez_dataset <- Ensembl_to_entrez(expr_set, dictionary)

## Script branch

In [296]:
dictionary <- Ensembl_mouse_dictionary_create(gset, org.Mm.egREFSEQ2EG)

In [297]:
Entrez_dataset <- Ensembl_to_entrez(gset, dictionary)

[1] "100038358"
[1] "100038591"
[1] "100038714"
[1] "100038746"
[1] "100042605"
[1] "100043424"
[1] "100043902"
[1] "100072"
[1] "100088"
[1] "100163"
[1] "100169"
[1] "100169864"
[1] "100177"
[1] "100213"
[1] "100273"
[1] "100317"
[1] "100417675"
[1] "100434"
[1] "100494"
[1] "100502698"
[1] "100502700"
[1] "100503044"
[1] "100503311"
[1] "100503495"
[1] "100503613"
[1] "100503659"
[1] "100503895"
[1] "100504464"
[1] "100515"
[1] "100737"
[1] "100764"
[1] "100855"
[1] "100929"
[1] "101023"
[1] "101100"
[1] "101437"
[1] "101488"
[1] "101497"
[1] "101502"
[1] "101540"
[1] "101592"
[1] "101631"
[1] "101772"
[1] "101867"
[1] "102022"
[1] "102032"
[1] "102098"
[1] "102103"
[1] "102323"
[1] "102334"
[1] "102448"
[1] "102545"
[1] "102566"
[1] "102580"
[1] "102747"
[1] "102857"
[1] "102866"
[1] "102991"
[1] "103012"
[1] "103080"
[1] "103098"
[1] "103135"
[1] "103175"
[1] "103220"
[1] "103236"
[1] "103268"
[1] "103573"
[1] "103583"
[1] "103836"
[1] "103841"
[1] "103844"
[1] "103967"
[1] "10396

In [306]:
Entrez_dataset

,GSM1647454,GSM1647455,GSM1647456,GSM1647457,GSM1647458,GSM1647459,GSM1647460,GSM1647461,GSM1647462,GSM1647463,GSM1647464,GSM1647465,GSM1647466,GSM1647467,GSM1647468,GSM1647469,GSM1647470,GSM1647471
100009600,5.267419,5.138059,5.519279,5.483820,5.523904,5.534099,5.534551,5.671926,5.610729,5.523904,5.254290,5.699321,5.358244,5.191436,5.566259,5.462666,5.363895,5.375659
100009609,3.143113,3.118565,3.112049,3.157388,3.243009,3.216725,3.157430,3.229067,3.241919,3.228338,3.181152,3.206978,3.228401,3.174369,3.011504,3.092957,3.069668,3.114338
100009614,5.946784,6.322555,6.243125,6.258440,6.228276,6.270767,6.367156,6.117669,5.996078,6.039930,6.112887,6.257987,6.092386,5.894801,5.674298,6.014341,5.981856,5.872741
100012,3.847308,3.839217,3.830725,3.940169,3.954303,3.821494,4.149675,3.847178,3.682562,3.827939,3.614376,3.814573,3.814008,3.639960,3.694231,3.472067,3.767664,3.698399
100017,8.327885,8.573890,8.474465,8.474648,8.107372,8.097307,7.692978,7.563667,7.425891,7.437565,7.756718,8.244728,8.515095,8.553017,8.920481,9.037358,8.393708,8.378303
100019,8.017983,7.996085,9.545509,9.548799,9.793167,9.836709,9.871809,9.883585,9.967563,9.882205,9.353433,9.490946,9.411304,9.340277,9.224262,9.221666,9.254988,9.226344
100033459,3.277588,3.628597,3.673823,3.422985,3.673804,3.154496,3.866785,3.055195,3.288391,3.197160,3.092615,3.404189,3.513875,3.260654,3.426166,3.209722,3.411561,2.942501
100034251,5.093351,5.217051,7.344791,7.270803,8.251543,8.352045,7.138493,6.800484,6.140188,5.761164,4.589360,4.473195,4.484169,4.454002,4.192492,4.223900,4.238418,4.259671
100034363,2.832139,2.783291,2.657024,2.789999,2.535577,2.495746,3.045378,2.943923,2.767741,2.773181,3.138489,3.243495,2.887596,3.230723,2.854909,2.868854,3.034933,2.946759
100036520,5.503308,5.455457,5.298147,5.382244,5.760710,5.555885,5.541282,5.595640,5.403751,5.763389,5.702404,5.421058,5.423803,5.447199,5.431789,5.541212,5.466559,5.369474


## saving

In [302]:
data <- Entrez_dataset #table of experiments
exp_data <- gset@phenoData@data #metadata with a description of columns

In [303]:
head(exp_data, n=3)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_institute,contact_address,contact_city,contact_zip/postal_code,contact_country,supplementary_file,data_row_count,cell type:ch1,day:ch1,genotype:ch1
,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>,<chr>
GSM1647454,2ndMEF_at_day0_rep1,GSM1647454,Public on Dec 31 2015,Mar 31 2015,Dec 31 2015,RNA,1,2nd mouse embryonic fibroblasts prior to induction,Mus musculus,"cell type: primary mouse embryonic fibroblast isolated from E13.5 tetraploid blastocyst complementation embryo,P4",⋯,Tongji University,"1239 Siping Road, Yangpu District",Shanghai,200092,China,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1647nnn/GSM1647454/suppl/GSM1647454_2ndMEF_rep1.CEL.gz,29643,"primary mouse embryonic fibroblast isolated from E13.5 tetraploid blastocyst complementation embryo,P4",0,Oct4-GFP/ Rosa26-M2rtTA transgenic background
GSM1647455,2ndMEF_at_day0_rep2,GSM1647455,Public on Dec 31 2015,Mar 31 2015,Dec 31 2015,RNA,1,2nd mouse embryonic fibroblasts prior to induction,Mus musculus,"cell type: primary mouse embryonic fibroblast isolated from E13.5 tetraploid blastocyst complementation embryo,P4",⋯,Tongji University,"1239 Siping Road, Yangpu District",Shanghai,200092,China,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1647nnn/GSM1647455/suppl/GSM1647455_2ndMEF_rep2.CEL.gz,29643,"primary mouse embryonic fibroblast isolated from E13.5 tetraploid blastocyst complementation embryo,P4",0,Oct4-GFP/ Rosa26-M2rtTA transgenic background
GSM1647456,Reprogramming_cells_at_day1_rep1,GSM1647456,Public on Dec 31 2015,Mar 31 2015,Dec 31 2015,RNA,1,reprogramming intermediate after 24hour of induction,Mus musculus,cell type: reprogramming intermediate,⋯,Tongji University,"1239 Siping Road, Yangpu District",Shanghai,200092,China,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1647nnn/GSM1647456/suppl/GSM1647456_D1_rep1.CEL.gz,29643,reprogramming intermediate,1,Oct4-GFP/ Rosa26-M2rtTA transgenic background


In [304]:
#Create directory for GSE files
dirname <- paste("./Data/", toupper(gse_name), sep="")
dir.create(dirname)

In [305]:
#save files
write.csv(data, paste(dirname, "/", gse_name, "_data.csv", sep=""))
write.csv(exp_data, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annotation_table, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE59527,  GSE21757- parse annotation

In [164]:
?getGEO

In [19]:
# load series and platform data from GEO
gse_name = "GSE21757"
gpl_name = "GPL6096"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)

GSE21757_series_matrix.txt.gz

Using locally cached version: /tmp/RtmpWuZhmx/GSE21757_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_double(),
  GSM542374 = col_double(),
  GSM542375 = col_double(),
  GSM542376 = col_double(),
  GSM542377 = col_double(),
  GSM542378 = col_double(),
  GSM542379 = col_double(),
  GSM542380 = col_double(),
  GSM542381 = col_double(),
  GSM542382 = col_double(),
  GSM542383 = col_double(),
  GSM542384 = col_double(),
  GSM542385 = col_double(),
  GSM542386 = col_double()
)


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL6nnn/GPL6096/annot/GPL6096.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

Using locally cached version of GPL6096 found here:
/t

In [100]:
annotation_table <- fData(gset)
colnames(annotation_table)

print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))

[1] "ID"              "GB_LIST"         "SPOT_ID"         "seqname"        
 [5] "RANGE_GB"        "RANGE_STRAND"    "RANGE_START"     "RANGE_STOP"     
 [9] "total_probes"    "gene_assignment" "mrna_assignment" "category"

[1] "Count of probe_ID:" "23332"             
[1] "Unique probe_ID:" "23332"           


In [101]:
# need to parse gene_assignment
annotation_table

,ID,GB_LIST,SPOT_ID,seqname,RANGE_GB,RANGE_STRAND,RANGE_START,RANGE_STOP,total_probes,gene_assignment,mrna_assignment,category
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
4305140,4305140,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4305147,4305147,,control,---,,---,0,0,4,---,--- // --- // pos_control // --- // --- // --- // --- // --- // ---,normgene->exon
4305425,4305425,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4305730,4305730,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4305790,4305790,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4305808,4305808,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4305936,4305936,,control,---,,---,0,0,1,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron
4306126,4306126,,Not currently mapped to latest genome,,,,NA,NA,NA,,,
4306193,4306193,,control,---,,---,0,0,4,---,--- // --- // neg_control // --- // --- // --- // --- // --- // ---,normgene->intron


In [286]:
#construct dataframe
annotation_table$ENTREZID <- NaN
probes <- rownames(annotation_table)
for (i in 1:length(probes)){ 
        #write ENTREZID at corresponding probes indices
        #e <- moex10stprobesetENTREZID[[probes[i]]] 
        s <- annotation_table$gene_assignment[[i]]

        if (nchar(s) > 3){
            #id <- strsplit(str_match(s, "// \\d* ///")[[1]], "//s*" )[[1]][[2]]
            id <- strsplit(s, "//")[[1]][[5]]
            id <- gsub(" ", "", id, fixed = TRUE)
            annotation_table$ENTREZID[[i]] <- id
        }
        
        
}

In [293]:
# This seems to be valid only for microarrays data with one old GPL

geneID = annotation_table$ENTREZID #take column with IDs
# Below we need delete those probe_IDs that map at several gene_ID;
# and sum those probe_ID expressions which map to one gene_ID
# create dataframe with geneID counts
geneID_count_table <- as.data.frame(table(geneID), stringsAsFactors=FALSE) 
#filter geneID count table
#take away those affymetrix probes which map to several genes, like 'Probe -> X///Y'
geneID_filt <- geneID_count_table[Reduce("&", 
                         list(!grepl('/', geneID_count_table$geneID), #sign '/' must NOT be in geneID
                         (nchar(geneID_count_table$geneID) > 0) )),   #length of geneID string must be > 0 
                          ] 

# Filter annotation table
annotation_table_filt <- annotation_table[annotation_table$ENTREZID %in% geneID_filt$geneID, ]

In [295]:
#take a table with expressions
ex <- exprs(gset)

#processed_ex <- ex[annotation_table_filt$ID, ] #if both indices numbers
processed_ex <- ex[as.character(annotation_table_filt$ID), ] #if they are strings

print(c("Before:", length(rownames(ex)), "After:", length(rownames(processed_ex))))
head(processed_ex)

[1] "Before:" "23332"   "After:"  "16681"  


,GSM542374,GSM542375,GSM542376,GSM542377,GSM542378,GSM542379,GSM542380,GSM542381,GSM542382,GSM542383,GSM542384,GSM542385,GSM542386
6747308,9.665384,9.609118,10.537270,10.285830,10.961560,10.342460,10.756670,10.698120,11.454420,10.652830,10.250100,10.664440,10.052690
6747309,11.856960,10.487090,11.866850,11.724620,11.602750,12.123230,11.556590,11.865150,12.248610,12.168860,10.912480,11.545250,10.937190
6747314,10.062460,9.961292,10.429860,9.738548,10.183630,9.942407,10.153990,10.053390,10.677130,10.334170,10.517440,10.011770,10.444040
6747326,3.197937,4.041018,3.936279,3.582773,3.814374,4.499316,4.121133,3.662272,5.045465,3.720603,3.730523,3.764731,4.127715
6747343,9.124737,9.634633,9.779807,9.123588,8.973856,8.846365,9.088464,8.934072,8.759983,8.269632,10.063960,8.613454,10.361740
6747354,3.826308,3.953627,3.792995,3.953627,4.166873,3.984015,3.953627,4.046142,3.953627,3.977552,3.953627,4.082313,3.950572


In [296]:
#check if all rownames in expr table and annot table are equal
if (all(rownames(processed_ex) == rownames(annotation_table_filt)))
{
    rownames(processed_ex) <- annotation_table_filt$ENTREZID
} else {
    print("Problem")
}

processed_ex_combined <- processed_ex 
print(c("Before:", length(rownames(processed_ex)), "After:", length(rownames(processed_ex_combined))))

[1] "Before:" "16681"   "After:"  "16681"  


In [299]:
data <- processed_ex_combined #table of experiments
exp_data <- gset@phenoData@data #metadata with a description of columns

In [300]:
#Create directory for GSE files
dirname <- paste("./Data/", toupper(gse_name), sep="")
dir.create(dirname)

In [301]:
#save files
write.csv(data, paste(dirname, "/", gse_name, "_data.csv", sep=""))
write.csv(exp_data, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annotation_table, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE48201, GSE46321 - auto annotation new

In [32]:
# load series and platform data from GEO
gse_name = "GSE46321"
gpl_name = "GPL13912"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]


Found 1 file(s)

GSE46321_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  .default = col_double()
)
ℹ Use `spec()` for the full column specifications.


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL13nnn/GPL13912/annot/GPL13912.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

File stored at: 

/tmp/Rtmplax3A4/GPL13912.soft



In [33]:
annotation_table <- fData(gset)
colnames(annotation_table)

print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))

[1] "ID"                   "COL"                  "ROW"                 
 [4] "NAME"                 "SPOT_ID"              "CONTROL_TYPE"        
 [7] "GB_ACC"               "GENE_ID"              "GENE_SYMBOL"         
[10] "GENE_NAME"            "UNIGENE_ID"           "ENSEMBL_ID"          
[13] "ACCESSION_STRING"     "CHROMOSOMAL_LOCATION" "CYTOBAND"            
[16] "DESCRIPTION"          "GO_ID"                "SEQUENCE"

[1] "Count of probe_ID:" "62976"             
[1] "Unique probe_ID:" "62976"           


In [34]:
annotation_table

,ID,COL,ROW,NAME,SPOT_ID,CONTROL_TYPE,GB_ACC,GENE_ID,GENE_SYMBOL,GENE_NAME,UNIGENE_ID,ENSEMBL_ID,ACCESSION_STRING,CHROMOSOMAL_LOCATION,CYTOBAND,DESCRIPTION,GO_ID,SEQUENCE
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,192,328,,GE_BrightCorner,pos,,NA,,,,,,,,,,
2,2,192,326,,DarkCorner,pos,,NA,,,,,,,,,,
3,3,192,324,,DarkCorner,pos,,NA,,,,,,,,,,
4,4,192,322,A_55_P2051983,,FALSE,NM_001001803,408198,Spink7,"serine peptidase inhibitor, Kazal type 7 (putative)",Mm.478742,ENSMUST00000076194,ref|NM_001001803|ens|ENSMUST00000076194|gb|BC148575|gb|BC156750,chr18:62753954-62753895,mm|18qE1,"Mus musculus serine peptidase inhibitor, Kazal type 7 (putative) (Spink7), mRNA [NM_001001803]",GO:0003674(molecular_function)|GO:0004867(serine-type endopeptidase inhibitor activity)|GO:0005575(cellular_component)|GO:0005576(extracellular region)|GO:0008150(biological_process)|GO:0030414(peptidase inhibitor activity),CAGTTTGTGGATCTGACTATATCACTTACGGGAATAAATGCAAGCTGTGTACAGAGATCT
5,5,192,320,A_52_P169082,,FALSE,NM_021294,13168,Dbil5,diazepam binding inhibitor-like 5,Mm.347413,,ref|NM_021294,chr11:76031955-76032014,mm|11qB5,"Mus musculus diazepam binding inhibitor-like 5 (Dbil5), mRNA [NM_021294]",GO:0000062(fatty-acyl-CoA binding)|GO:0005488(binding)|GO:0005737(cytoplasm)|GO:0006810(transport)|GO:0008289(lipid binding),TAAACAAAGGGATGTCCAAGATGGATGCCATGAGGATCTACATTGCTAAAGTGGAAGAGC
6,6,192,318,A_30_P01028193,,FALSE,,NA,,,,,chr14:47978277-47985238_F,chr14:47982421-47982480,mm|14qC1,lincRNA:chr14:47978277-47985238 forward strand,,CCATTAGAGCTGACTCCAAGCCATCAGTCCCAACGAGGAATGGTGAATTGGCTTCATCTA
7,7,192,316,A_52_P237997,,FALSE,NM_177274,320840,Negr1,neuronal growth regulator 1,Mm.317293,,ref|NM_177274,chr3:156834624-156834683,mm|3qH4,"Mus musculus neuronal growth regulator 1 (Negr1), transcript variant 2, mRNA [NM_177274]",GO:0003674(molecular_function)|GO:0005886(plasma membrane)|GO:0007155(cell adhesion)|GO:0008150(biological_process)|GO:0016020(membrane)|GO:0030425(dendrite)|GO:0031225(anchored to membrane)|GO:0043025(neuronal cell body),GTGGAGCTTCCATATCCAAAATCGTGCAACTGGGATTTTTCTACAGACACACAAGTTAGC
8,8,192,314,A_51_P414243,,FALSE,NM_153540,215494,C85492,expressed sequence C85492,Mm.402151,ENSMUST00000084743,ref|NM_153540|ens|ENSMUST00000084743|gb|AK042747|gb|AK133802,chr9:121890800-121890741,mm|9qF4,"Mus musculus expressed sequence C85492 (C85492), mRNA [NM_153540]","GO:0003674(molecular_function)|GO:0005575(cellular_component)|GO:0005576(extracellular region)|GO:0008150(biological_process)|GO:0016740(transferase activity)|GO:0016757(transferase activity, transferring glycosyl groups)",TAATCTTTGAATTTCATCTGTAGCTGAGTGACAGGTGTGTTTGGAGCAGTCATTAAATGA
9,9,192,312,A_55_P2136348,,FALSE,NM_007720,12776,Ccr8,chemokine (C-C motif) receptor 8,Mm.442098,ENSMUST00000048777,ref|NM_007720|ens|ENSMUST00000048777|gb|BC101966|gb|BC101967,chr9:120003293-120003352,mm|9qF4,"Mus musculus chemokine (C-C motif) receptor 8 (Ccr8), mRNA [NM_007720]",GO:0004871(signal transducer activity)|GO:0004872(receptor activity)|GO:0004930(G-protein coupled receptor activity)|GO:0004950(chemokine receptor activity)|GO:0005515(protein binding)|GO:0005886(plasma membrane)|GO:0006935(chemotaxis)|GO:0007165(signal transduction)|GO:0007186(G-protein coupled receptor protein signaling pathway)|GO:0016020(membrane)|GO:0016021(integral to membrane)|GO:0016493(C-C chemokine receptor activity),ATGTTCTTCATCACCCTAATGAGTGTGGACAGGTATCTGGCTATTGTCCACGCTGTCTAT


In [35]:
# This seems to be valid only for microarrays data with one old GPL

#take column with IDs
#geneID <- annotation_table$Entrez_gene 
geneID <- annotation_table$GENE_ID
# Below we need delete those probe_IDs that map at several gene_ID;
# and sum those probe_ID expressions which map to one gene_ID
# create dataframe with geneID counts
geneID_count_table <- as.data.frame(table(geneID), stringsAsFactors=FALSE) 
#filter geneID count table
geneID_filt <- geneID_count_table[Reduce("&", 
                         list(!grepl('/', geneID_count_table$geneID), #sign '/' must NOT be in geneID
                         (nchar(geneID_count_table$geneID) > 0) )),   #length of geneID string must be > 0 
                          ] 
#? should I somehow take into account X///Y genes?

# Filter annotation table
annotation_table_filt <- annotation_table[geneID %in% geneID_filt$geneID, ]

In [36]:
annotation_table_filt

,ID,COL,ROW,NAME,SPOT_ID,CONTROL_TYPE,GB_ACC,GENE_ID,GENE_SYMBOL,GENE_NAME,UNIGENE_ID,ENSEMBL_ID,ACCESSION_STRING,CHROMOSOMAL_LOCATION,CYTOBAND,DESCRIPTION,GO_ID,SEQUENCE
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
4,4,192,322,A_55_P2051983,,FALSE,NM_001001803,408198,Spink7,"serine peptidase inhibitor, Kazal type 7 (putative)",Mm.478742,ENSMUST00000076194,ref|NM_001001803|ens|ENSMUST00000076194|gb|BC148575|gb|BC156750,chr18:62753954-62753895,mm|18qE1,"Mus musculus serine peptidase inhibitor, Kazal type 7 (putative) (Spink7), mRNA [NM_001001803]",GO:0003674(molecular_function)|GO:0004867(serine-type endopeptidase inhibitor activity)|GO:0005575(cellular_component)|GO:0005576(extracellular region)|GO:0008150(biological_process)|GO:0030414(peptidase inhibitor activity),CAGTTTGTGGATCTGACTATATCACTTACGGGAATAAATGCAAGCTGTGTACAGAGATCT
5,5,192,320,A_52_P169082,,FALSE,NM_021294,13168,Dbil5,diazepam binding inhibitor-like 5,Mm.347413,,ref|NM_021294,chr11:76031955-76032014,mm|11qB5,"Mus musculus diazepam binding inhibitor-like 5 (Dbil5), mRNA [NM_021294]",GO:0000062(fatty-acyl-CoA binding)|GO:0005488(binding)|GO:0005737(cytoplasm)|GO:0006810(transport)|GO:0008289(lipid binding),TAAACAAAGGGATGTCCAAGATGGATGCCATGAGGATCTACATTGCTAAAGTGGAAGAGC
7,7,192,316,A_52_P237997,,FALSE,NM_177274,320840,Negr1,neuronal growth regulator 1,Mm.317293,,ref|NM_177274,chr3:156834624-156834683,mm|3qH4,"Mus musculus neuronal growth regulator 1 (Negr1), transcript variant 2, mRNA [NM_177274]",GO:0003674(molecular_function)|GO:0005886(plasma membrane)|GO:0007155(cell adhesion)|GO:0008150(biological_process)|GO:0016020(membrane)|GO:0030425(dendrite)|GO:0031225(anchored to membrane)|GO:0043025(neuronal cell body),GTGGAGCTTCCATATCCAAAATCGTGCAACTGGGATTTTTCTACAGACACACAAGTTAGC
8,8,192,314,A_51_P414243,,FALSE,NM_153540,215494,C85492,expressed sequence C85492,Mm.402151,ENSMUST00000084743,ref|NM_153540|ens|ENSMUST00000084743|gb|AK042747|gb|AK133802,chr9:121890800-121890741,mm|9qF4,"Mus musculus expressed sequence C85492 (C85492), mRNA [NM_153540]","GO:0003674(molecular_function)|GO:0005575(cellular_component)|GO:0005576(extracellular region)|GO:0008150(biological_process)|GO:0016740(transferase activity)|GO:0016757(transferase activity, transferring glycosyl groups)",TAATCTTTGAATTTCATCTGTAGCTGAGTGACAGGTGTGTTTGGAGCAGTCATTAAATGA
9,9,192,312,A_55_P2136348,,FALSE,NM_007720,12776,Ccr8,chemokine (C-C motif) receptor 8,Mm.442098,ENSMUST00000048777,ref|NM_007720|ens|ENSMUST00000048777|gb|BC101966|gb|BC101967,chr9:120003293-120003352,mm|9qF4,"Mus musculus chemokine (C-C motif) receptor 8 (Ccr8), mRNA [NM_007720]",GO:0004871(signal transducer activity)|GO:0004872(receptor activity)|GO:0004930(G-protein coupled receptor activity)|GO:0004950(chemokine receptor activity)|GO:0005515(protein binding)|GO:0005886(plasma membrane)|GO:0006935(chemotaxis)|GO:0007165(signal transduction)|GO:0007186(G-protein coupled receptor protein signaling pathway)|GO:0016020(membrane)|GO:0016021(integral to membrane)|GO:0016493(C-C chemokine receptor activity),ATGTTCTTCATCACCCTAATGAGTGTGGACAGGTATCTGGCTATTGTCCACGCTGTCTAT
10,10,192,310,A_51_P108228,,FALSE,NM_007390,11441,Chrna7,"cholinergic receptor, nicotinic, alpha polypeptide 7",Mm.113464,ENSMUST00000032738,ref|NM_007390|ens|ENSMUST00000032738|gb|AK157809|gb|L37663,chr7:70244064-70244005,mm|7qC,"Mus musculus cholinergic receptor, nicotinic, alpha polypeptide 7 (Chrna7), mRNA [NM_007390]","GO:0000187(activation of MAPK activity)|GO:0001540(beta-amyloid binding)|GO:0001666(response to hypoxia)|GO:0001988(positive regulation of heart rate involved in baroreceptor response to decreased systemic arterial blood pressure)|GO:0004872(receptor activity)|GO:0004889(nicotinic acetylcholine-activated cation-selective channel activity)|GO:0005216(ion channel activity)|GO:0005230(extracellular ligand-gated ion channel activity)|GO:0005515(protein binding)|GO:0005624(membrane fraction)|GO:0005737(cytoplasm)|GO:

In [37]:
#take a table with expressions
ex <- exprs(gset)

processed_ex <- ex[annotation_table_filt$ID, ] #if both indices numbers or text
#processed_ex <- ex[as.character(annotation_table_filt$ID), ] #if one no

print(c("Before:", length(rownames(ex)), "After:", length(rownames(processed_ex))))
head(processed_ex)

[1] "Before:" "62976"   "After:"  "33093"  


,GSM1128619,GSM1128620,GSM1128621,GSM1128622,GSM1128623,GSM1128624,GSM1128625,GSM1128626,GSM1128627,GSM1128628,⋯,GSM1359916,GSM1359917,GSM1359918,GSM1359919,GSM1359920,GSM1359921,GSM1359922,GSM1359923,GSM1359924,GSM1359925
4,6.847307,6.561031,6.406002,6.435739,6.301756,6.455957,6.424704,6.678935,6.364185,6.703581,⋯,6.528239,6.351944,6.623651,6.394721,6.379604,6.368713,6.325356,6.491262,6.209233,6.235734
5,8.153137,8.034532,7.220075,7.148738,7.267475,7.152202,7.683249,7.904296,7.783811,8.315311,⋯,7.336354,7.371499,7.786538,7.649621,7.716506,7.735560,7.617606,7.722306,7.527976,7.495528
7,6.529955,6.527334,6.287950,6.488969,6.470213,6.521718,6.281761,6.479007,6.203542,6.609259,⋯,6.527016,6.831566,6.409746,6.563992,6.579640,6.494493,6.405590,6.462064,6.391427,6.290030
8,12.029919,11.946343,11.603969,11.515852,12.866122,12.840276,12.834901,13.078771,11.762318,11.180461,⋯,10.792275,10.798413,11.272631,11.292682,10.755180,10.800740,11.082137,11.283259,10.937608,10.945531
9,6.382525,7.406313,6.458011,6.717143,6.356417,6.425082,6.014505,6.449429,6.273072,6.527460,⋯,6.705033,6.643480,6.455927,6.593732,6.474873,6.369154,6.497673,6.477565,6.383340,6.218306
10,6.650058,6.680433,6.394094,6.926413,6.404855,6.723433,6.491526,6.748965,6.576860,7.017907,⋯,6.533697,6.450372,6.701243,6.779057,6.547377,6.675433,6.432924,6.750451,6.466755,6.482924


In [38]:
#check if all rownames in expr table and annot table are equal
if (all(rownames(processed_ex) == rownames(annotation_table_filt)))
{
    rownames(processed_ex) <- annotation_table_filt$GENE_ID
} else {
    print("Problem")
}
# aggregation with sum
processed_ex_combined <- processed_ex#rowsum(processed_ex, row.names(processed_ex))
print(c("Before:", length(rownames(processed_ex)), "After:", length(rownames(processed_ex_combined))))

[1] "Before:" "33093"   "After:"  "33093"  


In [39]:
head(processed_ex_combined)

,GSM1128619,GSM1128620,GSM1128621,GSM1128622,GSM1128623,GSM1128624,GSM1128625,GSM1128626,GSM1128627,GSM1128628,⋯,GSM1359916,GSM1359917,GSM1359918,GSM1359919,GSM1359920,GSM1359921,GSM1359922,GSM1359923,GSM1359924,GSM1359925
408198,6.847307,6.561031,6.406002,6.435739,6.301756,6.455957,6.424704,6.678935,6.364185,6.703581,⋯,6.528239,6.351944,6.623651,6.394721,6.379604,6.368713,6.325356,6.491262,6.209233,6.235734
13168,8.153137,8.034532,7.220075,7.148738,7.267475,7.152202,7.683249,7.904296,7.783811,8.315311,⋯,7.336354,7.371499,7.786538,7.649621,7.716506,7.735560,7.617606,7.722306,7.527976,7.495528
320840,6.529955,6.527334,6.287950,6.488969,6.470213,6.521718,6.281761,6.479007,6.203542,6.609259,⋯,6.527016,6.831566,6.409746,6.563992,6.579640,6.494493,6.405590,6.462064,6.391427,6.290030
215494,12.029919,11.946343,11.603969,11.515852,12.866122,12.840276,12.834901,13.078771,11.762318,11.180461,⋯,10.792275,10.798413,11.272631,11.292682,10.755180,10.800740,11.082137,11.283259,10.937608,10.945531
12776,6.382525,7.406313,6.458011,6.717143,6.356417,6.425082,6.014505,6.449429,6.273072,6.527460,⋯,6.705033,6.643480,6.455927,6.593732,6.474873,6.369154,6.497673,6.477565,6.383340,6.218306
11441,6.650058,6.680433,6.394094,6.926413,6.404855,6.723433,6.491526,6.748965,6.576860,7.017907,⋯,6.533697,6.450372,6.701243,6.779057,6.547377,6.675433,6.432924,6.750451,6.466755,6.482924


In [40]:
data <- processed_ex_combined #table of experiments
exp_data <- gset@phenoData@data #metadata with a description of columns

In [41]:
head(exp_data, n=3)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,supplementary_file,data_row_count,time point:ch1,tissue:ch1,treatment:ch1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM1128619,"18hrs pulse B cells, time 0, technical replicate 1",GSM1128619,Public on Nov 17 2013,Apr 23 2013,Mar 31 2014,RNA,1,aT0,Mus musculus,tissue: primary B cells,⋯,185 Cambridge Street,Boston,Ma,02114,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1128nnn/GSM1128619/suppl/GSM1128619_07_IG_2300252800513234_Cy3_aT0_2_3.txt.gz,62976,0 hrs,primary B cells,B cells pulsed for 18hrs with C/EBPa
GSM1128620,"18hrs pulse B cells, time 0, technical replicate 2",GSM1128620,Public on Nov 17 2013,Apr 23 2013,Mar 31 2014,RNA,1,aT0,Mus musculus,tissue: primary B cells,⋯,185 Cambridge Street,Boston,Ma,02114,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1128nnn/GSM1128620/suppl/GSM1128620_39_IG_2300252800515286_Cy3_aT0_2_3.txt.gz,62976,0 hrs,primary B cells,B cells pulsed for 18hrs with C/EBPa
GSM1128621,"18hrs pulse B cells, time 48, technical replicate 1",GSM1128621,Public on Nov 17 2013,Apr 23 2013,Mar 31 2014,RNA,1,aT48,Mus musculus,tissue: primary B cells,⋯,185 Cambridge Street,Boston,Ma,02114,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM1128nnn/GSM1128621/suppl/GSM1128621_09_IG_1100252800513235_Cy3_aT48_1_1.txt.gz,62976,48 hrs,primary B cells,"B cells pulsed for 18hrs with C/EBPa, induced for 48 hrs with OSKM"


In [42]:
#Create directory for GSE files
dirname <- paste("./Data/", toupper(gse_name), sep="")
dir.create(dirname)

Warning message in dir.create(dirname):
“'./Data/GSE46321' already exists”


In [43]:
#save files
write.csv(data, paste(dirname, "/", gse_name, "_data.csv", sep=""))
write.csv(exp_data, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annotation_table, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE71255, GSE10871, GSE28688, GSE35229, GSE37896, GSE116309, GSE81891, GSE38509 - auto annotation old


**Problems**
* 5241 probe ID are without Entrez ID in GPL auto-annotation
* What should be done with probe ID mapping to several EntrezID, like X///Y, espesially when those genes are similar

In [11]:
# load series and platform data from GEO
gse_name = "gse10871"
gpl_name = "GPL1261"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]


Found 1 file(s)

GSE10871_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  ID_REF = col_character()
)
ℹ Use `spec()` for the full column specifications.


File stored at: 

/tmp/Rtmpk6wods/GPL1261.annot.gz

Warning message:
“64 parsing failures.
  row             col           expected    actual         file
45038 Platform_SPOTID 1/0/T/F/TRUE/FALSE --Control literal data
45039 Platform_SPOTID 1/0/T/F/TRUE/FALSE --Control literal data
45040 Platform_SPOTID 1/0/T/F/TRUE/FALSE --Control literal data
45041 Platform_SPOTID 1/0/T/F/TRUE/FALSE --Control literal data
45042 Platform_SPOTID 1/0/T/F/TRUE/FALSE --Control literal data
..... ............... .................. ......... ............
See problems(...) for more details.
”


In [12]:
annotation_table <- fData(gset)
colnames(annotation_table)

print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))

[1] "ID"                    "Gene title"            "Gene symbol"          
 [4] "Gene ID"               "UniGene title"         "UniGene symbol"       
 [7] "UniGene ID"            "Nucleotide Title"      "GI"                   
[10] "GenBank Accession"     "Platform_CLONEID"      "Platform_ORF"         
[13] "Platform_SPOTID"       "Chromosome location"   "Chromosome annotation"
[16] "GO:Function"           "GO:Process"            "GO:Component"         
[19] "GO:Function ID"        "GO:Process ID"         "GO:Component ID"

[1] "Count of probe_ID:" "45101"             
[1] "Unique probe_ID:" "45101"           


In [13]:
head(annotation_table)

,ID,Gene title,Gene symbol,Gene ID,UniGene title,UniGene symbol,UniGene ID,Nucleotide Title,GI,GenBank Accession,⋯,Platform_ORF,Platform_SPOTID,Chromosome location,Chromosome annotation,GO:Function,GO:Process,GO:Component,GO:Function ID,GO:Process ID,GO:Component ID
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<lgl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1415670_at,1415670_at,"coatomer protein complex, subunit gamma 1",Copg1,54161,,,,"Mus musculus coatomer protein complex, subunit gamma, mRNA (cDNA clone MGC:30335 IMAGE:3992144), complete cds",19354080,BC024686,⋯,NA,NA,6 39.13 cM,"Chromosome 6, NC_000072.6 (87887819..87913595)",structural molecule activity,establishment of Golgi localization///intracellular protein transport///organelle transport along microtubule///protein transport///transport///vesicle-mediated transport,COPI vesicle coat///Golgi apparatus///Golgi membrane///cell///cytoplasm///cytoplasmic vesicle///intracellular///membrane///membrane coat,GO:0005198,GO:0051683///GO:0006886///GO:0072384///GO:0015031///GO:0006810///GO:0016192,GO:0030126///GO:0005794///GO:0000139///GO:0005623///GO:0005737///GO:0031410///GO:0005622///GO:0016020///GO:0030117
1415671_at,1415671_at,"ATPase, H+ transporting, lysosomal V0 subunit D1",Atp6v0d1,11972,,,,"Mus musculus ATPase, H+ transporting, lysosomal V0 subunit D1 (Atp6v0d1), mRNA",141802838,NM_013477,⋯,NA,NA,8 D3,"Chromosome 8, NC_000074.6 (105524470..105566040, complement)","hydrogen ion transmembrane transporter activity///hydrogen-exporting ATPase activity, phosphorylative mechanism///protein complex binding",ATP hydrolysis coupled proton transport///cell projection organization///ion transport///proton transport///transport,"apical plasma membrane///axon terminus///colocalizes_with centrosome///early endosome///extracellular exosome///lysosomal membrane///membrane///neuron projection///protein complex///proton-transporting V-type ATPase, V0 domain///synaptic vesicle///vacuolar proton-transporting V-type ATPase complex",GO:0015078///GO:0008553///GO:0032403,GO:0015991///GO:0030030///GO:0006811///GO:0015992///GO:0006810,GO:0016324///GO:0043679///colocalizes_with GO:0005813///GO:0005769///GO:0070062///GO:0005765///GO:0016020///GO:0043005///GO:0043234///GO:0033179///GO:0008021///GO:0016471
1415672_at,1415672_at,"golgi autoantigen, golgin subfamily a, 7",Golga7,57437,,,,"Mus musculus golgi autoantigen, golgin subfamily a, 7 (Golga7), transcript variant 1, mRNA",109809742,NM_020585,⋯,NA,NA,8 A2,"Chromosome 8, NC_000074.6 (23241326..23257434, complement)",contributes_to protein-cysteine S-palmitoyltransferase activity,Golgi to plasma membrane protein transport///Golgi to plasma membrane protein transport///Golgi to plasma membrane transport///peptidyl-L-cysteine S-palmitoylation///peptidyl-L-cysteine S-palmitoylation///protein stabilization///protein targeting to membrane,Golgi apparatus///Golgi membrane///Golgi stack///Golgi stack///extracellular exosome///intrinsic component of Golgi membrane///membrane///palmitoyltransferase complex///palmitoyltransferase complex,contributes_to GO:0019706,GO:0043001///GO:0043001///GO:0006893///GO:0018230///GO:0018230///GO:0050821///GO:0006612,GO:0005794///GO:0000139///GO:0005795///GO:0005795///GO:0070062///GO:0031228///GO:0016020///GO:0002178///GO:0002178
1415673_at,1415673_at,phosphoserine phosphatase,Psph,100678,,,,"Mus musculus phosphoserine phosphatase (Psph), mRNA",270265804,NM_133900,⋯,NA,NA,5 G1.3,"Chromosome 5, NC_000071.6 (129765558..129787253, complement)",calcium ion binding///hydrolase activity///magnesium ion binding///magnesium ion binding///metal ion binding///phosphatase activity///phosphoserine phosphatase activity///phosphoserine phosphatase activity///protein homodimerization activity,L-serine biosynthetic process///L-serine metabolic process///cellular amino acid biosynthetic process///metabolic process,cytoplasm///neuron projection,GO:0005509///GO:0016787///GO:0000287///GO:0000287///GO:0046872///G

In [14]:
# This seems to be valid only for microarrays data with one old GPL

geneID = annotation_table$`Gene ID` #take column with IDs
# Below we need delete those probe_IDs that map at several gene_ID;
# and sum those probe_ID expressions which map to one gene_ID
# create dataframe with geneID counts
geneID_count_table <- as.data.frame(table(geneID), stringsAsFactors=FALSE) 
#filter geneID count table
#take away those affymetrix probes which map to several genes, like 'Probe -> X///Y'
geneID_filt <- geneID_count_table[Reduce("&", 
                         list(!grepl('/', geneID_count_table$geneID), #sign '/' must NOT be in geneID
                         (nchar(geneID_count_table$geneID) > 0) )),   #length of geneID string must be > 0 
                          ] 

# Filter annotation table
annotation_table_filt <- annotation_table[annotation_table$`Gene ID` %in% geneID_filt$geneID, ]

In [13]:
#take a table with expressions
ex <- exprs(gset)

#processed_ex <- ex[annotation_table_filt$ID, ] #if both indices numbers
processed_ex <- ex[as.character(annotation_table_filt$ID), ] #if they are strings

print(c("Before:", length(rownames(ex)), "After:", length(rownames(processed_ex))))
head(processed_ex)

[1] "Before:" "45101"   "After:"  "38664"  


,GSM275544,GSM275546,GSM275547,GSM275548,GSM275551,GSM275552,GSM275554,GSM275555,GSM275556,GSM275557,⋯,GSM275572,GSM275573,GSM275574,GSM275575,GSM275576,GSM275577,GSM275578,GSM275579,GSM275580,GSM275581
1415670_at,660.822,685.387,611.015,585.653,584.786,644.169,755.767,717.892,702.217,713.311,⋯,1445.630,1211.320,1162.790,1253.820,856.066,796.842,1187.520,864.721,660.508,650.859
1415671_at,638.648,648.144,769.181,812.415,685.714,647.907,782.244,710.628,636.893,567.480,⋯,1177.470,1129.070,1012.480,1036.800,1000.460,1039.860,1305.000,1286.290,729.169,715.419
1415672_at,1876.190,1706.550,1616.720,1609.580,1881.010,1601.780,1460.160,1572.520,1865.070,1568.020,⋯,1449.220,1586.530,1907.610,2152.310,1587.000,1630.370,2264.070,2209.290,1965.460,1720.900
1415673_at,217.298,232.434,261.754,209.446,366.954,397.160,387.046,365.566,400.182,424.585,⋯,1254.390,1193.090,812.680,789.066,341.248,349.608,352.214,363.987,973.340,921.844
1415674_a_at,738.994,720.872,787.971,740.081,897.291,804.731,828.232,834.185,760.671,767.783,⋯,773.325,778.433,866.206,839.185,727.993,737.271,806.742,732.198,686.369,630.492
1415675_at,284.446,266.126,253.392,226.374,682.554,496.266,489.516,420.154,543.902,479.565,⋯,351.734,377.648,490.140,559.023,533.112,505.218,522.695,455.541,286.607,281.148


In [14]:
#check if all rownames in expr table and annot table are equal
if (all(rownames(processed_ex) == rownames(annotation_table_filt)))
{
    rownames(processed_ex) <- annotation_table_filt$`Gene ID`
} else {
    print("Problem")
}

processed_ex_combined <- processed_ex 
print(c("Before:", length(rownames(processed_ex)), "After:", length(rownames(processed_ex_combined))))

[1] "Before:" "38664"   "After:"  "38664"  


In [15]:
head(processed_ex_combined)

,GSM275544,GSM275546,GSM275547,GSM275548,GSM275551,GSM275552,GSM275554,GSM275555,GSM275556,GSM275557,⋯,GSM275572,GSM275573,GSM275574,GSM275575,GSM275576,GSM275577,GSM275578,GSM275579,GSM275580,GSM275581
54161,660.822,685.387,611.015,585.653,584.786,644.169,755.767,717.892,702.217,713.311,⋯,1445.630,1211.320,1162.790,1253.820,856.066,796.842,1187.520,864.721,660.508,650.859
11972,638.648,648.144,769.181,812.415,685.714,647.907,782.244,710.628,636.893,567.480,⋯,1177.470,1129.070,1012.480,1036.800,1000.460,1039.860,1305.000,1286.290,729.169,715.419
57437,1876.190,1706.550,1616.720,1609.580,1881.010,1601.780,1460.160,1572.520,1865.070,1568.020,⋯,1449.220,1586.530,1907.610,2152.310,1587.000,1630.370,2264.070,2209.290,1965.460,1720.900
100678,217.298,232.434,261.754,209.446,366.954,397.160,387.046,365.566,400.182,424.585,⋯,1254.390,1193.090,812.680,789.066,341.248,349.608,352.214,363.987,973.340,921.844
60409,738.994,720.872,787.971,740.081,897.291,804.731,828.232,834.185,760.671,767.783,⋯,773.325,778.433,866.206,839.185,727.993,737.271,806.742,732.198,686.369,630.492
13481,284.446,266.126,253.392,226.374,682.554,496.266,489.516,420.154,543.902,479.565,⋯,351.734,377.648,490.140,559.023,533.112,505.218,522.695,455.541,286.607,281.148


In [16]:
data <- processed_ex_combined #table of experiments
exp_data <- gset@phenoData@data #metadata with a description of columns

In [17]:
head(exp_data, n=3)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_institute,contact_address,contact_city,contact_state,contact_zip/postal_code,contact_country,supplementary_file,supplementary_file.1,data_row_count,relation
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM275544,Partially reprogrammed cell line MCV8 (SSEA+) replicate 1,GSM275544,Public on May 28 2008,Mar 17 2008,Aug 28 2018,RNA,1,MCV8,Mus musculus,"SSEA1 positive, GFP negative",⋯,Broad Institute,7 Cambridge Center,Cambridge,MA,02142,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275544/suppl/GSM275544.CEL.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275544/suppl/GSM275544.CHP.gz,45101,Reanalyzed by: GSE119085
GSM275546,Partially reprogrammed cell line MCV8 (SSEA+) replicate 2,GSM275546,Public on May 28 2008,Mar 17 2008,Aug 28 2018,RNA,1,MCV8,Mus musculus,"SSEA1 positive, GFP negative",⋯,Broad Institute,7 Cambridge Center,Cambridge,MA,02142,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275546/suppl/GSM275546.CEL.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275546/suppl/GSM275546.CHP.gz,45101,Reanalyzed by: GSE119085
GSM275547,Partially reprogrammed cell line MCV8 (SSEA-) replicate 1,GSM275547,Public on May 28 2008,Mar 17 2008,Aug 28 2018,RNA,1,MCV8,Mus musculus,"SSEA1 negative, GFP negative",⋯,Broad Institute,7 Cambridge Center,Cambridge,MA,02142,USA,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275547/suppl/GSM275547.CEL.gz,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM275nnn/GSM275547/suppl/GSM275547.CHP.gz,45101,Reanalyzed by: GSE119085


In [18]:
#Create directory for GSE files
dirname <- paste("./Data/", toupper(gse_name), sep="")
dir.create(dirname)

Warning message in dir.create(dirname):
“'./Data/GSE10871' already exists”


In [19]:
# data <- exprs(gset)
# exp_data <- pData(gset)
# annotation_table <- fData(gset)

In [20]:
#save files
write.csv(data, paste(dirname, "/", gse_name, "_data.csv", sep=""))
write.csv(exp_data, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annotation_table, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE115451

In [8]:
# load series and platform data from GEO
gse_name = "GSE115451"
gpl_name = "GPL21273"

gset <- getGEO(gse_name)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)

GSE115451_series_matrix.txt.gz

Using locally cached version: /tmp/RtmpMIgwuA/GSE115451_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM3178498 = col_character(),
  GSM3178499 = col_character(),
  GSM3178500 = col_character(),
  GSM3178501 = col_character(),
  GSM3178502 = col_character(),
  GSM3178503 = col_character(),
  GSM3178504 = col_character(),
  GSM3178505 = col_character(),
  GSM3178506 = col_character(),
  GSM3178507 = col_character(),
  GSM3178508 = col_character(),
  GSM3178509 = col_character(),
  GSM3983470 = col_character(),
  GSM3983471 = col_character(),
  GSM3983472 = col_character(),
  GSM3983473 = col_character(),
  GSM3983474 = col_character(),
  GSM3983475 = col_character()
)


Using locally cached version of GPL21273 found here:
/tmp/RtmpMIgwuA/GPL21273.soft 



In [10]:
gset

ExpressionSet (storageMode: lockedEnvironment)
assayData: 0 features, 18 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: GSM3178498 GSM3178499 ... GSM3983475 (18 total)
  varLabels: title geo_accession ... time:ch1 (49 total)
  varMetadata: labelDescription
featureData: none
experimentData: use 'experimentData(object)'
  pubMedIds: 32016422 
Annotation: GPL21273 

In [9]:
annotation_table <- fData(gset)
colnames(annotation_table)

print(c("Count of probe_ID:", length(rownames(annotation_table))))
print(c("Unique probe_ID:", length(unique(rownames(annotation_table)))))

character(0)

[1] "Count of probe_ID:" "0"                 
[1] "Unique probe_ID:" "0"               


# GSE114581

In [5]:
### 2 ready
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE114581
gse_name = "GSE114581"
#Platforms (3)	
#GPL8321	[Mouse430A_2] Affymetrix Mouse Genome 430A 2.0 Array
#GPL13112	Illumina HiSeq 2000 (Mus musculus)
#GPL19057	Illumina NextSeq 500 (Mus musculus)

GSE114581 <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
#GSE114581_GPL13112 <- GSE114581$`GSE114581-GPL13112_series_matrix.txt.gz`
#GSE114581_GPL19057 <- GSE114581$`GSE114581-GPL19057_series_matrix.txt.gz`
GSE114581_GPL8321 <- GSE114581$`GSE114581-GPL8321_series_matrix.txt.gz`
GSE114581_GPL8321_mt <- exprs(GSE114581_GPL8321)
GSE114581_GPL8321_pheno <- phenoData(GSE114581_GPL8321) #get data table 
GSE114581_GPL8321_annot <- fData(GSE114581_GPL8321)

Found 3 file(s)

GSE114581-GPL13112_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  .default = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL13nnn/GPL13112/annot/GPL13112.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

File stored at: 

/tmp/RtmplY8eJI/GPL13112.soft

GSE114581-GPL19057_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM4667681 = col_character(),
  GSM4667682 = col_character(),
  GSM4667683 = col_character(),
  GSM4667684 = col_character(),
  GSM4667685 = col_character()
)


Warning message in download.file(myurl, destfil

In [6]:
GSE114581_GPL8321_mt

,GSM3145292,GSM3145293,GSM3145294,GSM3145295,GSM3145296,GSM3145297,GSM3145298,GSM3145299,GSM3145300,GSM3145301,⋯,GSM3145308,GSM3145309,GSM3145310,GSM3145311,GSM3145312,GSM3145313,GSM3145314,GSM3145315,GSM3145316,GSM3145317
1415670_at,2429.6824,2387.3983,2708.5451,2576.9717,2554.4327,2654.8246,2330.6941,2425.4313,1901.5316,1616.4695,⋯,1564.3270,1588.7026,1626.6281,1027.1110,1276.1264,1962.4493,1751.6634,1386.07313,1482.6812,1418.5714
1415671_at,4782.8022,4910.3662,4424.8094,5339.7682,4966.4237,4566.3005,4489.8094,2803.5307,2367.0065,2670.3238,⋯,2702.3393,1609.4011,1926.9635,2355.1949,1953.1039,1813.9854,2210.2437,2124.50718,1571.4320,1362.5431
1415672_at,3420.2870,3547.9003,3165.3325,3731.2025,3633.2870,3916.3425,3807.1704,4911.7588,3965.7799,5278.7746,⋯,3606.3478,3456.2093,4252.7052,3558.5124,3249.3149,3015.0450,3937.8176,3496.33033,3334.4971,3468.9672
1415673_at,691.0945,764.9766,607.1906,886.9247,972.0888,1069.5158,858.6964,1051.7405,872.8229,895.1453,⋯,1436.3704,743.9334,947.9779,1221.8058,1228.7908,931.2477,810.4498,661.75831,1145.3196,518.4192
1415674_a_at,2137.5219,1937.1289,1870.3737,1805.2579,1677.5749,2042.4024,1713.9620,1572.2116,1104.2146,1093.5711,⋯,1637.1277,1130.3238,1294.5447,1602.1162,943.8308,986.1880,914.5673,781.71844,849.5676,779.8982
1415675_at,929.1179,1019.0168,844.3379,923.6640,886.7023,1091.5824,1004.2768,788.6219,726.0658,749.5769,⋯,957.9022,510.9124,624.1339,464.3326,544.6342,459.0307,397.9751,521.55631,587.4331,445.5513
1415676_a_at,3602.8765,3710.7157,3492.1491,3712.9437,3834.1837,3875.0744,3824.2978,3266.9167,2593.0894,2045.3777,⋯,2756.4834,2696.8580,3051.6190,3534.4002,4608.9367,2956.4899,2780.1090,2623.40789,3186.4703,2360.1923
1415677_at,473.6089,403.0578,605.8430,565.1877,584.9498,533.4441,503.1460,449.6312,389.7548,422.0470,⋯,715.8108,693.0556,731.6512,219.0052,111.9045,350.4605,366.8009,411.18866,643.8958,939.7448
1415678_at,1120.1903,1065.9034,1151.3660,1128.6063,1103.3571,1082.3438,1149.9198,1102.4824,1167.9060,1215.4078,⋯,1256.7650,1304.8581,1119.3180,1082.8187,1535.9271,993.6093,1092.2153,949.26473,1221.9490,991.1211
1415679_at,2543.3292,2888.9716,2208.1768,2750.8980,2795.4729,2411.2869,3004.5805,1746.8895,2174.0664,1907.0710,⋯,2416.0721,2474.7786,2161.6673,2686.8330,1551.0280,1736.0015,1952.2628,2013.59053,1919.9227,1712.7810


In [2]:
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL8321
GSE114581_GPL8321_entrez_annot <- read.csv2("GPL8321-17396.txt", 
                                            comment.char = "#", 
                                            header = T, as.is = T, sep = "\t")
#change names to entrez ids
GSE114581_GPL8321_df <- as.data.frame(GSE114581_GPL8321_mt)
GSE114581_GPL8321_df$ids_old <- rownames(GSE114581_GPL8321_df)
GSE114581_GPL8321_df <- merge.data.frame(GSE114581_GPL8321_df, 
                                         GSE114581_GPL8321_entrez_annot[c("ENTREZ_GENE_ID", "ID")],
                                         by.x = "ids_old", by.y = "ID", all.x = T)

#there are 13614 ENTREZ ids and 22690 native ids
length(unique(GSE114581_GPL8321_df$ENTREZ_GENE_ID))
sum(is.na(GSE114581_GPL8321_df$ENTREZ_GENE_ID)) #0
length(unique(GSE114581_GPL8321_df$ids_old))

ERROR: Error in getGEO(gse_name, GSEMatrix = TRUE, getGPL = TRUE, AnnotGPL = TRUE): could not find function "getGEO"


In [33]:
### 7 ready, but a lot of na in entrez ids
gse_name = "GSE81891"
GSE81891 <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
GSE81891_df <- as.data.frame(exprs(GSE81891[[1]]))
GSE81891_annot <- fData(GSE81891[[1]])
GSE81891_pheno <- phenoData(GSE81891[[1]])

Found 1 file(s)

GSE81891_series_matrix.txt.gz


── Column specification ──────────────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM2177032 = col_double(),
  GSM2177033 = col_double(),
  GSM2177034 = col_double(),
  GSM2177035 = col_double(),
  GSM2177036 = col_double(),
  GSM2177037 = col_double(),
  GSM2177038 = col_double(),
  GSM2177039 = col_double(),
  GSM2177040 = col_double(),
  GSM2177041 = col_double(),
  GSM2177042 = col_double()
)


File stored at: 

/tmp/RtmplY8eJI/GPL10558.annot.gz

Warning message:
“13 parsing failures.
  row     col               expected                   actual         file
29686 Gene ID no trailing characters 100616668///283507       literal data
29841 Gene ID no trailing characters 267004///2074            literal data
30401 Gene ID no trailing characters 100303453///27185///7257 literal data
31184 Gene ID no trailing characters 100507670///57592        literal data
31271 Gene ID no trailing characters 

In [46]:
sum(!is.na(GSE81891_annot[rownames(GSE81891_df), ][['Gene ID']]))

[1] 31253

# GSE89455 (Human)

In [ ]:
# load series and platform data from GEO
gse_name = "GSE89455"
gpl_name = "GPL13607"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

In [ ]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annot)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

In [ ]:
rownames(exp) = annot$GeneName

In [ ]:
library(mygene)
names1 <- unique(rownames(exp))
query <- as.data.frame(queryMany(names1, scopes="symbol", species="human"))
query <- query[is.na(query$notfound),]
query <- query[!is.na(query$entrezgene),]

In [ ]:
todrop <- c()
for (i in 1:nrow(query)){
    s <- query[i,]$query
    m <- nrow(query[query$query == s,])
    #cond <- sum(is.na(query[query$query==s,]$entrezgene))
    if (m > 1){
        todrop <- c(todrop, s)
    }
}


In [ ]:
symbols <- setdiff(unique(query$query), todrop)
etz <- query[query$query %in% symbols,]$entrezgene

In [ ]:
data <- exp[symbols,]
rownames(data) <- etz

In [ ]:
dirname <- 'data'
#save files
write.csv(data, paste(dirname, "/", gse_name, "_data.csv", sep=""))
write.csv(pheno, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annot, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE50206

In [11]:
# load series and platform data from GEO
gse_name = "GSE50206"
gpl_name = "GPL14550"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 2 file(s)

GSE50206-GPL10787_series_matrix.txt.gz

Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM1330506 = col_double(),
  GSM1330507 = col_double(),
  GSM1330508 = col_double(),
  GSM1330509 = col_double(),
  GSM1330510 = col_double(),
  GSM1330511 = col_double(),
  GSM1330512 = col_double(),
  GSM1330513 = col_double(),
  GSM1330514 = col_double(),
  GSM1330515 = col_double(),
  GSM1330516 = col_double(),
  GSM1330517 = col_double(),
  GSM1330518 = col_double(),
  GSM1330519 = col_double(),
  GSM1330520 = col_double()
)

Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL10nnn/GPL10787/annot/GPL10787.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

File stored at: 

/tmp/Rtmp5B35d7/GPL10787.soft

Warning message:
“255525 parsing failures.
  row       

In [12]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annot)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

[1] "ID"                   "SPOT_ID"              "CONTROL_TYPE"        
 [4] "REFSEQ"               "GB_ACC"               "GENE"                
 [7] "GENE_SYMBOL"          "GENE_NAME"            "UNIGENE_ID"          
[10] "ENSEMBL_ID"           "TIGR_ID"              "ACCESSION_STRING"    
[13] "CHROMOSOMAL_LOCATION" "CYTOBAND"             "DESCRIPTION"         
[16] "GO_ID"                "SEQUENCE"

[1] "Number of samples:" "169"               
[1] "Count of probe_ID:" "42536"             
[1] "Unique probe_ID:" "42536"           


In [48]:
sub <- annot[!is.na(annot$GENE), ]
data <- exp[rownames(sub),]

rownames(data) <- as.character(sub$GENE)

In [89]:
dirname <- 'data'
#save files
write.csv(data, paste(dirname, "/", gse_name, "_rawdata.csv", sep=""))
write.csv(pheno, paste(dirname, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annot, paste(dirname, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE54848

In [3]:
# load series and platform data from GEO
gse_name = "GSE54848"
gpl_name = "GPL14550"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 2 file(s)

GSE54848-GPL13534_series_matrix.txt.gz

Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

See spec(...) for full column specifications.

Warning message in download.file(myurl, destfile, mode = mode, quiet = TRUE, method = getOption("download.file.method.GEOquery")):
“cannot open URL 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPL13nnn/GPL13534/annot/GPL13534.annot.gz': HTTP status was '404 Not Found'”
Annotation GPL not available, so will use submitter GPL instead

File stored at: 

/tmp/RtmpMF3HjE/GPL13534.soft

Warning message:
“65 parsing failures.
   row     col           expected     actual         file
485513 SPOT_ID 1/0/T/F/TRUE/FALSE rs10796216 literal data
485514 SPOT_ID 1/0/T/F/TRUE/FALSE rs715359   literal data
485515 SPOT_ID 1/0/T/F/TRUE/FALSE rs1040870  literal data
485516 SPOT_ID 1/0/T/F/TRUE/FALSE rs10936224 literal data
485517 SPOT_ID 1/0/T/F/TRUE/FALSE rs213028   literal data
...... ....... ..............

In [4]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annot)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

[1] "ID"                   "SPOT_ID"              "CONTROL_TYPE"        
 [4] "REFSEQ"               "GB_ACC"               "GENE"                
 [7] "GENE_SYMBOL"          "GENE_NAME"            "UNIGENE_ID"          
[10] "ENSEMBL_ID"           "TIGR_ID"              "ACCESSION_STRING"    
[13] "CHROMOSOMAL_LOCATION" "CYTOBAND"             "DESCRIPTION"         
[16] "GO_ID"                "SEQUENCE"

[1] "Number of samples:" "107"               
[1] "Count of probe_ID:" "42545"             
[1] "Unique probe_ID:" "42545"           


In [11]:
sub <- annot[!is.na(annot$GENE), ]
data <- exp[rownames(sub),]

rownames(data) <- as.character(sub$GENE)

In [14]:
dirname <- 'data'
#save files
write.csv(data, paste(dirname, "/", gse_name, "/", gse_name, "_rawdata.csv", sep=""))
write.csv(pheno, paste(dirname, "/", gse_name, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annot, paste(dirname, "/", gse_name, "/", gse_name, "_probe_annotation.csv", sep=""))

# GSE28688

In [16]:
# load series and platform data from GEO
gse_name = "GSE28688"
gpl_name = "GPL6883"

gset <- getGEO(gse_name, GSEMatrix=TRUE, getGPL=TRUE, AnnotGPL=TRUE)
if (length(gset) > 1) idx <- grep(gpl_name, attr(gset, "names")) else idx <- 1
gset <- gset[[idx]]

Found 1 file(s)

GSE28688_series_matrix.txt.gz

Parsed with column specification:
cols(
  ID_REF = col_character(),
  GSM710513 = col_double(),
  GSM710514 = col_double(),
  GSM710515 = col_double(),
  GSM710516 = col_double(),
  GSM710517 = col_double(),
  GSM710518 = col_double(),
  GSM710519 = col_double(),
  GSM710520 = col_double(),
  GSM710521 = col_double(),
  GSM710522 = col_double(),
  GSM710523 = col_double(),
  GSM710524 = col_double(),
  GSM710525 = col_double(),
  GSM710526 = col_double()
)

File stored at: 

/tmp/RtmpMF3HjE/GPL6883.annot.gz



In [17]:
annot <- fData(gset) #annotation
exp <- exprs(gset) #expression
pheno <- pData(gset) #phenotype

colnames(annot)

print(c("Number of samples:", ncol(exp)))
print(c("Count of probe_ID:", length(rownames(annot))))
print(c("Unique probe_ID:", length(unique(rownames(annot)))))

[1] "ID"                    "Gene title"            "Gene symbol"          
 [4] "Gene ID"               "UniGene title"         "UniGene symbol"       
 [7] "UniGene ID"            "Nucleotide Title"      "GI"                   
[10] "GenBank Accession"     "Platform_CLONEID"      "Platform_ORF"         
[13] "Platform_SPOTID"       "Chromosome location"   "Chromosome annotation"
[16] "GO:Function"           "GO:Process"            "GO:Component"         
[19] "GO:Function ID"        "GO:Process ID"         "GO:Component ID"      
[22] "Platform_SEQUENCE"

[1] "Number of samples:" "14"                
[1] "Count of probe_ID:" "24526"             
[1] "Unique probe_ID:" "24526"           


In [34]:
sub <- annot[!is.na(annot['Gene ID']), ]
data <- exp[rownames(sub),]

rownames(data) <- as.character(sub[,'Gene ID'])

In [36]:
dirname <- 'data'
#save files
write.csv(data, paste(dirname, "/", gse_name, "/", gse_name, "_rawdata.csv", sep=""))
write.csv(pheno, paste(dirname, "/", gse_name, "/", gse_name, "_metadata.csv", sep=""))
write.csv(annot, paste(dirname, "/", gse_name, "/", gse_name, "_probe_annotation.csv", sep=""))